In [1]:
from __future__ import print_function

In [2]:
import keras
import scipy
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras import layers

In [3]:
batch_size = 32 
num_classes = 10
epochs = 100
data_augmentation = True

In [4]:
cifar_data = cifar10.load_data()
# type(cifar_data)

In [5]:
(X_train, y_train), (X_test, y_test) = cifar_data

In [6]:
print('X_train shape:', X_train.shape)

X_train shape: (50000, 32, 32, 3)


In [7]:
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

50000 train samples
10000 test samples


***Convert class vectors to binary class matrices.***

In [8]:
y_train = keras.utils.to_categorical(y_train, num_classes)

In [9]:
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
X_train.shape[1:]

(32, 32, 3)

***Create a Sequential model***

In [11]:
model = keras.Sequential()

2023-07-29 11:07:39.834110: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-29 11:07:39.834197: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ejaj): /proc/driver/nvidia/version does not exist


In [12]:
# Convolutional Block 1
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=X_train.shape[1:]))

In [13]:
# Convolutional Block 2
model.add(layers.Conv2D(48, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

In [14]:
# Convolutional Block 3
model.add(layers.Conv2D(80, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

In [15]:
# Convolutional Block 4
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dropout(0.25))

In [16]:
# Fully Connected Block
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dropout(0.25))

In [17]:
# Output Layer
model.add(layers.Dense(num_classes, activation='softmax'))

In [18]:
# Initiate the optimizer
opt = keras.optimizers.Adam(lr=0.0001)

/home/kazi/Projects/Python/CIFAR10Vision/venv/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [19]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 48)        13872     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 48)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 80)        34640     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 80)         0         
 2D)                                                    

In [21]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [22]:
tb_callback = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
import scipy
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(
        X_train, 
        y_train, 
        batch_size=batch_size, 
        epochs=epochs, 
        validation_data=(X_test, y_test), 
        shuffle=True, 
        callbacks=[tb_callback]
    )
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False  # randomly flip images
    ) 
    datagen.fit(X_train)
    model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          steps_per_epoch=X_train.shape[0] // batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          callbacks=[tb_callback])

Using real-time data augmentation.
Epoch 1/100
1562/1562 [==============================] - 291s 185ms/step - loss: 2.0416 - accuracy: 0.2257 - val_loss: 1.8292 - val_accuracy: 0.3299
Epoch 2/100
1562/1562 [==============================] - 278s 178ms/step - loss: 1.7998 - accuracy: 0.3241 - val_loss: 1.6934 - val_accuracy: 0.3850
Epoch 3/100
1562/1562 [==============================] - 271s 173ms/step - loss: 1.7018 - accuracy: 0.3640 - val_loss: 1.5942 - val_accuracy: 0.4303
Epoch 4/100
1562/1562 [==============================] - 279s 179ms/step - loss: 1.6283 - accuracy: 0.3999 - val_loss: 1.4951 - val_accuracy: 0.4564
Epoch 5/100
1562/1562 [==============================] - 271s 174ms/step - loss: 1.5589 - accuracy: 0.4272 - val_loss: 1.4528 - val_accuracy: 0.4725
Epoch 6/100
1562/1562 [==============================] - 272s 174ms/step - loss: 1.5034 - accuracy: 0.4527 - val_loss: 1.3602 - val_accuracy: 0.5101
Epoch 7/100
1562/1562 [==============================] - 272s 174ms/ste

In [ ]:
scores=model.evaluate(x_test,y_test,verbose=1)
print("Test Loss",scores[0])
print("Test Accuracy",scores[1])